In [ ]:
!nvidia-smi

Sat Jan  9 08:55:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install tape_proteins >> /dev/null

In [ ]:
!pip install bert-for-tf2 >> /dev/null

In [ ]:
import os
import math
import datetime
import random
import importlib

# from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm

import pandas as pd
import numpy as np

import re

import tensorflow as tf
from tensorflow import keras
from  tensorflow.keras.callbacks import EarlyStopping

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import torch
from tape import ProteinBertModel, TAPETokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/My Drive/Colab Notebooks NU"

/content/drive/.shortcut-targets-by-id/1FqifPUlm6cgaaWF0H2AQqipYJL1XcCct/Colab Notebooks


In [ ]:
import bert_thesis_experiments.utils as utils
import bert_thesis_experiments.my_models as my_models

In [ ]:
seqfile = 'DeepDom_Code/DeepDom-master/processed_seq.txt'; #file name of the processed sequence data (output from dataprocess.pl)
labelfile= 'DeepDom_Code/DeepDom-master/processed_label.txt'; #file name of the processed label data (output from dataprocess.pl)

In [ ]:
(ids,seqs) = utils.process_inputseqs(seqfile)
(ids,labels) = utils.process_inputlabels(labelfile)

In [ ]:
rawdata = list(zip(seqs,labels))
random.shuffle(rawdata)

In [ ]:
model = ProteinBertModel.from_pretrained('bert-base')
tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model

In [ ]:
sequence = rawdata[50][0]
seq_length = len(sequence)
print(sequence)
# token_ids = torch.tensor([np.append(tokenizer.encode(sequence), [0]*3)]) # [0] is pad symbol id
seq_wo_dash = sequence[:len(sequence) if sequence.find("-") == -1 else sequence.find("-")]
if seq_length == len(seq_wo_dash):
  token_ids = torch.tensor([tokenizer.encode(seq_wo_dash[1:-1])])
else:
  token_ids = torch.tensor([np.append(tokenizer.encode(seq_wo_dash), [0]*(seq_length-len(seq_wo_dash)-2))], dtype=int)
print(token_ids)
output = model(token_ids)
sequence_output = output[0]
pooled_output = output[1]

# NOTE: pooled_output is *not* trained for the transformer, do not use
# w/o fine-tuning. A better option for now is to simply take a mean of
# the sequence output

TFLTIPIRLHEPNEFKLRLQEFTFQAFEFNERVEYQFKKSKTNELESPVLSKDFNTTDSNALVDNGIIPSEFLLYLVSMKHKLATNCAIYEVSMKAAIAFHDYELAMTTWKNRGKFRMTDAFQKLTPGERQQSDATFAQLMVEYFTNEKMYQDALSIILSSLKTVNWQYSMVKSLHRALLAIEDENSAARLLSVVNRKSK
tensor([[ 2, 10, 15, 23, 13, 19, 13, 21, 15, 12,  9, 19, 17,  9, 10, 14, 15, 21,
         15, 20,  9, 10, 23, 10, 20,  5, 10,  9, 10, 17,  9, 21, 25,  9, 28, 20,
         10, 14, 14, 22, 14, 23, 17,  9, 15,  9, 22, 19, 25, 15, 22, 14,  8, 10,
         17, 23, 23,  8, 22, 17,  5, 15, 25,  8, 17, 11, 13, 13, 19, 22,  9, 10,
         15, 15, 28, 15, 25, 22, 16, 14, 12, 14, 15,  5, 23, 17,  7,  5, 13, 28,
          9, 25, 22, 16, 14,  5,  5, 13,  5, 10, 12,  8, 28,  9, 15,  5, 16, 23,
         23, 26, 14, 17, 21, 11, 14, 10, 21, 16, 23,  8,  5, 10, 20, 14, 15, 23,
         19, 11,  9, 21, 20, 20, 22,  8,  5, 23, 10,  5, 20, 15, 16, 25,  9, 28,
         10, 23, 17,  9, 14, 16, 28, 20,  8,  5, 15, 22, 13, 13, 15, 22, 22, 15,
         14, 23, 25, 17, 26, 20, 28, 22, 16, 25, 14, 22, 15, 12, 21,  

In [ ]:
print(sequence)
print(len(sequence))
print(token_ids)
print(len(list(token_ids[0])))
print(sequence_output.shape)
print(pooled_output.shape)
# print(pooled_output)
print(pooled_output[0].tolist())
# print(type(output[0][0].detach().numpy()))
print(output[0][0].detach().numpy()[:, :64].shape)
print(output[0][0].detach().numpy().shape)

TFLTIPIRLHEPNEFKLRLQEFTFQAFEFNERVEYQFKKSKTNELESPVLSKDFNTTDSNALVDNGIIPSEFLLYLVSMKHKLATNCAIYEVSMKAAIAFHDYELAMTTWKNRGKFRMTDAFQKLTPGERQQSDATFAQLMVEYFTNEKMYQDALSIILSSLKTVNWQYSMVKSLHRALLAIEDENSAARLLSVVNRKSK
200
tensor([[ 2, 10, 15, 23, 13, 19, 13, 21, 15, 12,  9, 19, 17,  9, 10, 14, 15, 21,
         15, 20,  9, 10, 23, 10, 20,  5, 10,  9, 10, 17,  9, 21, 25,  9, 28, 20,
         10, 14, 14, 22, 14, 23, 17,  9, 15,  9, 22, 19, 25, 15, 22, 14,  8, 10,
         17, 23, 23,  8, 22, 17,  5, 15, 25,  8, 17, 11, 13, 13, 19, 22,  9, 10,
         15, 15, 28, 15, 25, 22, 16, 14, 12, 14, 15,  5, 23, 17,  7,  5, 13, 28,
          9, 25, 22, 16, 14,  5,  5, 13,  5, 10, 12,  8, 28,  9, 15,  5, 16, 23,
         23, 26, 14, 17, 21, 11, 14, 10, 21, 16, 23,  8,  5, 10, 20, 14, 15, 23,
         19, 11,  9, 21, 20, 20, 22,  8,  5, 23, 10,  5, 20, 15, 16, 25,  9, 28,
         10, 23, 17,  9, 14, 16, 28, 20,  8,  5, 15, 22, 13, 13, 15, 22, 22, 15,
         14, 23, 25, 17, 26, 20, 28, 22, 16, 25, 14, 22, 15, 12, 2

In [ ]:
def convert_input_to_TapeBertPoolingOutput(sequence, tokenizer, model):
  max_sequence_length = len(sequence)
  seq_wo_dash = sequence[:len(sequence) if sequence.find("-") == -1 else sequence.find("-")]
  if max_sequence_length == len(seq_wo_dash):
    token_ids = torch.tensor([tokenizer.encode(seq_wo_dash[1:-1])])
  else:
    token_ids = torch.tensor([np.append(tokenizer.encode(seq_wo_dash), [0]*(max_sequence_length-len(seq_wo_dash)-2))], dtype=int)
  
  output = model(token_ids)
  # sequence_output = output[0]
  # pooled_output = output[1]

  return output[1][0].tolist()

In [ ]:
def convert_input_to_TapeBertEmbedding(sequence, tokenizer, model):
  max_sequence_length = len(sequence)
  seq_wo_dash = sequence[:len(sequence) if sequence.find("-") == -1 else sequence.find("-")]
  if max_sequence_length == len(seq_wo_dash):
    token_ids = torch.tensor([tokenizer.encode(seq_wo_dash[1:-1])])
  else:
    token_ids = torch.tensor([np.append(tokenizer.encode(seq_wo_dash), [0]*(max_sequence_length-len(seq_wo_dash)-2))], dtype=int)
  
  output = model(token_ids)
  # sequence_output = output[0]
  # pooled_output = output[1]

  return output[0][0].detach().numpy()

In [ ]:
inputX = []
if os.path.exists("bert_thesis_experiments/tape_protein_files/train_seq_bert_iupac_tape_embedding.npy"):
  inputX = np.load("bert_thesis_experiments/tape_protein_files/train_seq_bert_iupac_tape_embedding.npy")
else:
  for i in tqdm(rawdata, ncols="600px"):
    inputX.append(convert_input_to_TapeBertEmbedding(i[0], tokenizer, model))
  # inputX = [convert_input_to_TapeBertPoolingOutput(i[0], tokenizer, model) for i in rawdata]

  np.save('bert_thesis_experiments/tape_protein_files/train_seq_bert_iupac_tape_embedding.npy', inputX)


In [ ]:
print(np.array(inputX).shape)
# print(np.array(inputX[0]).reshape(-1,768).shape)
# np.array(inputX[1])

In [ ]:
if os.path.exists("bert_thesis_experiments/tape_protein_files/train_label_bert_iupac_tape_embedding.npy"):
  inputY = np.load("bert_thesis_experiments/tape_protein_files/train_label_bert_iupac_tape_embedding.npy")
else:
  # inputY = [utils.convertlabels_to_binary(i[1]) for i in rawdata]
  inputY = [utils.convertlabels_to_categorical(i[1]) for i in rawdata]
  np.save('bert_thesis_experiments/tape_protein_files/train_label_bert_iupac_tape_embedding.npy', inputY)

In [ ]:
print(np.array(inputY).shape)

In [ ]:
def get_train_X_Y_Val_ffnn_tape_pool(inputX, inputY):
  data=list(zip(inputX,inputY))
  random.seed(4)
  random.shuffle(data)
  random.shuffle(data)
      
  train_num=int(len(inputX)*0.9)
  train=data[0:train_num]
  val=data[train_num:]

  trainX=np.array([i[0] for i in train])
  trainY=np.array([i[1] for i in train])
  # xx=np.dstack(trainX)
  # xx=np.rollaxis(xx,-1)

  print(trainX.shape)   
  print(trainY.shape) 

  # yy=np.dstack(trainY)
  # yy=np.rollaxis(yy,-1)

  # print(yy.shape)
      
  valX=np.array([i[0] for i in val])
  valY=np.array([i[1] for i in val])
  # valX=np.dstack(valX)
  # valX=np.rollaxis(valX,-1)

  # valY=np.dstack(valY)
  # valY=np.rollaxis(valY,-1)

  vv=(valX,valY)

  print("\n---Validation Data---")
  print(valX.shape)
  print(valY.shape)

  return trainX, trainY, vv

In [ ]:
trainX, trainY, vv = get_train_X_Y_Val_ffnn_tape_pool(inputX, inputY)

(28260, 768)
(28260, 200)

---Validation Data---
(3140, 768)
(3140, 200)


In [ ]:
def create_model_tape_ffnn_pool(input_dim, classes):
  model = keras.models.Sequential()

  model.add(keras.layers.Dense(units=1500, input_dim=input_dim))
  model.add(keras.layers.Activation("relu"))
  model.add(keras.layers.Dropout(0.5))

  for i in range(0,3):
      model.add(keras.layers.Dense(units=1500, input_dim=1500))
      model.add(keras.layers.Activation("relu"))
  model.add(keras.layers.Dropout(0.5))

  model.add(keras.layers.Dense(units=classes, input_dim=1500))
  model.add(keras.layers.Activation("sigmoid"))

  return model

In [ ]:
model = create_model_tape_ffnn_pool(768, seq_length)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              1153500   
_________________________________________________________________
activation (Activation)      (None, 1500)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1500)              2251500   
_________________________________________________________________
activation_1 (Activation)    (None, 1500)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1500)              2251500   
_________________________________________________________________
activation_2 (Activation)    (None, 1500)              0

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=0.01), metrics=["binary_accuracy"])
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adadelta(lr=.01), metrics=['binary_accuracy'])

fitHistory_batch = model.fit(x=trainX, y=trainY, validation_data=vv, epochs=300, batch_size=128, verbose=1)

Epoch 1/300
221/221 [==============================] - 1s 5ms/step - loss: 0.6635 - binary_accuracy: 0.6138 - val_loss: 0.6208 - val_binary_accuracy: 0.7723
Epoch 2/300
221/221 [==============================] - 1s 4ms/step - loss: 0.5733 - binary_accuracy: 0.7547 - val_loss: 0.5097 - val_binary_accuracy: 0.8158
Epoch 3/300
221/221 [==============================] - 1s 4ms/step - loss: 0.5108 - binary_accuracy: 0.8046 - val_loss: 0.4780 - val_binary_accuracy: 0.8191
Epoch 4/300
221/221 [==============================] - 1s 4ms/step - loss: 0.4991 - binary_accuracy: 0.8138 - val_loss: 0.4745 - val_binary_accuracy: 0.8191
Epoch 5/300
221/221 [==============================] - 1s 4ms/step - loss: 0.4950 - binary_accuracy: 0.8162 - val_loss: 0.4736 - val_binary_accuracy: 0.8191
Epoch 6/300
221/221 [==============================] - 1s 4ms/step - loss: 0.4926 - binary_accuracy: 0.8174 - val_loss: 0.4734 - val_binary_accuracy: 0.8191
Epoch 7/300
221/221 [==============================] - 1s 

KeyboardInterrupt: ignored